In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re # regex
from urllib.parse import urlparse, parse_qs # urlparse & parse_qs
from scipy.stats import entropy # Cho url_entropy ở

In [ ]:
path = '/kaggle/input/labeldata/normal_labeled.log'

In [ ]:
import re
from typing import Union, Iterable, List, Dict, Optional
import pandas as pd

# ──────────────────────────────────────────────────────────────
# 1) Pre-compile regex (Nginx / Apache “combined” format)
#    - STRICT   : có dấu " quanh request + referrer + user-agent
#    - FALLBACK : thiếu hoặc hỏng dấu " (bắt thoáng hơn)
# ──────────────────────────────────────────────────────────────
NGINX_REGEX_STRICT = re.compile(
    r'(?P<ip>\S+)\s+-\s+-\s+'                              # IP - -
    r'\[(?P<timestamp>[^\]]+)]\s+'                         # [timestamp]
    r'"(?P<method>[A-Z]+)\s+'                              # "METHOD␣
    r'(?P<url>.+?)\s+'                                     # URL (non-greedy)
    r'(?P<protocol>[A-Z]+/\d(?:\.\d)?)"\s+'                # PROTOCOL"
    r'(?P<status>\d{3}|-)\s+'                              # status
    r'(?P<size>\d+|-)\s+'                                  # size
    r'"(?P<referrer>[^"]*)"\s+'                            # "referrer"
    r'"(?P<user_agent>[^"]*)"'                            # "user-agent"
    r'(?:[ \t]+(?P<label>[01]))?$',        #  ← thêm nhóm label tuỳ chọn
    flags=re.IGNORECASE,
)

NGINX_REGEX_FALLBACK = re.compile(
    r'(?P<ip>\S+)\s+-\s+-\s+'                              # IP - -
    r'\[(?P<timestamp>[^\]]+)]\s+'                         # [timestamp]
    r'(?P<method>[A-Z]+)\s+'                               # METHOD
    r'(?P<url>.+?)\s+'                                     # URL
    r'(?P<protocol>[A-Z]+/\d(?:\.\d)?)\s+'                 # PROTOCOL
    r'(?P<status>\d{3}|-)\s+'                              # status
    r'(?P<size>\d+|-)\s+'                                  # size
    r'(?P<referrer>\S+|-)\s+'                              # referrer (không quotes)
    r'(?P<user_agent>.+)'                                 # user-agent (còn lại)
    r'(?:[ \t]+(?P<label>[01]))?$',        #  ← thêm nhóm label tuỳ chọn

    flags=re.IGNORECASE,
)

# Gộp thành tuple để lần lượt thử
NGINX_COMBINED_PATTERNS = (NGINX_REGEX_STRICT, NGINX_REGEX_FALLBACK)

# ──────────────────────────────────────────────────────────────
# 2) Tiện ích: loại bỏ ký tự control (nếu log bị lẫn \x00 …)
# ──────────────────────────────────────────────────────────────
def strip_control(s: str) -> str:
    """Remove leading control chars (0x00–0x1F) ở đầu dòng."""
    return re.sub(r'^[\x00-\x1F]+', "", s)

# ──────────────────────────────────────────────────────────────
# 3) Hàm wrapper parse_nginx_log
# ──────────────────────────────────────────────────────────────
def parse_nginx_log(
    source: Union[str, Iterable[str]],
    patterns: Iterable[re.Pattern] = NGINX_COMBINED_PATTERNS,
    as_dataframe: bool = True,
    encoding: Optional[str] = "utf-8",
) -> Union[pd.DataFrame, List[Dict[str, str]]]:
    """
    Parse log Nginx / Apache (combined) thành list[dict] hoặc pandas.DataFrame.

    Args:
        source (str | Iterable[str]):
            • Chuỗi đường dẫn file, hoặc
            • Iterable (list, generator, ...) các dòng log.
        patterns (Iterable[re.Pattern]): Danh sách regex sẽ thử lần lượt.
        as_dataframe (bool): True -> trả về DataFrame, False -> list[dict].
        encoding (str | None): Encoding khi mở file (nếu source là path).

    Returns:
        pandas.DataFrame | list[dict]
    """
    # 1) Lấy iterator dòng log
    if isinstance(source, str):                # truyền path
        fh = open(source, "r", encoding=encoding, errors="replace")
        lines = fh
        close_file = True
    else:                                      # iterable dòng
        lines = source
        close_file = False

    # 2) Parse
    parsed: List[Dict[str, str]] = []
    for raw_line in lines:
        line = strip_control(raw_line.rstrip("\n"))
        for pat in patterns:
            m = pat.match(line)
            if m:
                parsed.append(m.groupdict())
                break                          # matched → sang dòng kế
        # nếu muốn ghi lại MISS, thêm else: missed.append(line)

    # 3) Đóng file nếu cần
    if close_file:
        fh.close()

    # 4) Trả kết quả
    return pd.DataFrame(parsed) if as_dataframe else parsed

In [ ]:
# stores output in parsed_log.csv
import pandas as pd
df = parse_nginx_log(path)

In [ ]:
df.dropna(subset=['label'], inplace=True)
df['label'] = df['label'].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42,
    stratify=df['label'] # Rất quan trọng để giữ tỷ lệ label trong cả 2 tập
)

print("Kích thước tập Train:", train_df.shape)
print("Kích thước tập Test:", test_df.shape)

In [ ]:
df.isna().sum()/len(df)

In [ ]:
df.sample(5)

## Time-based feature handling

In [ ]:
def timestamp_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Nhận vào một DataFrame chứa cột 'timestamp' và trả về DataFrame
    đã được bổ sung đầy đủ các feature về thời gian.
    """
    df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
    
    df = df.sort_values('timestamp_dt').reset_index(drop=True)

    df['hour_of_day'] = df['timestamp_dt'].dt.hour
    df['day_of_week'] = df['timestamp_dt'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

    def get_part_of_day(hour):
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'
    df['part_of_day'] = df['hour_of_day'].apply(get_part_of_day)

    df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df['time_since_last_event'] = df['timestamp_dt'].diff().dt.total_seconds().fillna(0)
    df.drop(columns=["timestamp_dt"], inplace=True)
    df.drop(columns=["timestamp"], inplace=True)

    return df

In [ ]:
df = timestamp_features(df)
df.sample(5)

# Feature extraction process

## URL Feature handling

## URL Suspicious Patterns

In [ ]:
import pandas as pd
import re
from urllib.parse import unquote_plus
import base64

# ==============================================================================
# HÀM PHÂN TÍCH CUỐI CÙNG (DANH SÁCH PATTERN ĐÃ ĐƯỢC DỌN DẸP)
# ==============================================================================
def final_url_analyzer(url_string: str) -> tuple:
    """
    Hàm cuối cùng, với danh sách pattern đã được dọn dẹp và sửa lỗi.
    """
    # --- BỘ QUY TẮC ĐÃ ĐƯỢC TINH GỌN VÀ SỬA LỖI ---
    patterns = [
        # --- SQL Injection ---
        r'(union\s+select)',
        r'(select\s+.*\s+from)',
        r'(insert\s+into)',
        r'(delete\s+from)',
        r'(drop\s+table)',
        r'(--|#|\/\*|;\s*--)',
        r'(load_file\s*\()',
        r'(information_schema\.)',
        r'(pg_sleep|waitfor\s+delay|sleep|benchmark)\s*\(',
        r'(xp_cmdshell)',
        # <<< Sửa lỗi cho log #7: bắt OR '1'='1 và các dạng tương tự >>>
        r"(?:'|\")\s*or\s+(?:'|\")?.+?(?:'|\")?\s*=\s*(?:'|\")?.+?(?:'|\")?",

        # --- XSS ---
        r'(<script)',
        r'(<iframe)',
        r'(<svg)',
        r'(<img\s+[^>]*src\s*=\s*[\'"]?javascript:)',
        r'(on(error|load|mouseover)\s*=)',
        r'(eval\s*\()',
        r'(document\.cookie)',
        r'(alert\s*\()',

        # --- Path Traversal & File Inclusion ---
        r'(\.\.\/|\.\.\\|%2e%2e|%c0%ae)',
        r'(etc\/(passwd|shadow))',
        r'(proc\/(self|environ))',
        r'(boot\.ini|win\.ini)',
        r'(php|file|data):\/\/',

        # --- Command Injection ---
        r'(&&|;|\||`|\$\()',
        r'(\b(cat|ls|whoami|id|wget|curl|bash|sh|cmd)\s+)',
        r'(\/bin\/(ba)?sh)',
    ]
    combined_pattern = re.compile('|'.join(patterns), re.IGNORECASE)

    strings_to_check = set()
    try:
        strings_to_check.add(url_string)
        decoded_url = unquote_plus(unquote_plus(url_string))
        strings_to_check.add(decoded_url)
    except:
        decoded_url = url_string

    # Tách các phần và thử decode hex/base64
    parts = re.split(r'[=,&;/?]', decoded_url)
    for part in parts:
        part = part.strip()
        if len(part) < 4:
            continue
        # hex
        try:
            if all(c in '0123456789abcdefABCDEF' for c in part) and len(part) % 2 == 0:
                strings_to_check.add(bytes.fromhex(part).decode('utf-8', 'ignore'))
        except:
            pass
        # base64
        try:
            missing_padding = len(part) % 4
            if missing_padding:
                part += '=' * (4 - missing_padding)
            strings_to_check.add(base64.b64decode(part).decode('utf-8', 'ignore'))
        except:
            pass

    # Quét từng chuỗi
    for text in strings_to_check:
        match = combined_pattern.search(text)
        if match:
            return 1

    return 0



In [ ]:
df[df[['url', 'method', 'protocol']].isnull().any(axis=1)]['url']


In [ ]:
df[['is_suspicious']] = df['url'].apply(lambda x: pd.Series(final_url_analyzer(x)))
df

## User Agent Features

In [ ]:
from user_agents import parse

def calculate_entropy(text_string: str) -> float:
    """
    Tính entropy của chuỗi ký tự (dựa trên xác suất xuất hiện ký tự).
    """
    import math
    from collections import Counter

    if not text_string:
        return 0.0

    counts = Counter(text_string)
    total = len(text_string)
    entropy = -sum((count / total) * math.log2(count / total) for count in counts.values())
    return entropy


def user_agent_features(df):
    """
    Thêm các đặc trưng liên quan đến User-Agent vào DataFrame đầu vào.
    """
    df = df.copy()  # tránh tác động trực tiếp

    df['ua_parsed'] = df['user_agent'].astype(str).apply(parse)

    # 1. Trình duyệt (Browser Family)
    df['ua_browser_family'] = df['ua_parsed'].apply(lambda ua: ua.browser.family)

    # 2. Phiên bản trình duyệt (Major Version)
    df['ua_browser_version_major'] = df['ua_parsed'].apply(lambda ua: ua.browser.version[0] if ua.browser.version else None)

    # 3. Hệ điều hành (OS Family)
    df['ua_os_family'] = df['ua_parsed'].apply(lambda ua: ua.os.family)

    # 4. Phiên bản hệ điều hành (Major Version)
    df['ua_os_version_major'] = df['ua_parsed'].apply(lambda ua: ua.os.version[0] if ua.os.version else None)

    # 5. Thiết bị (Device Family/Brand)
    df['ua_device_family'] = df['ua_parsed'].apply(lambda ua: ua.device.family)
    df['ua_device_brand'] = df['ua_parsed'].apply(lambda ua: ua.device.brand)

    # 6-10. Các flag nhận diện thiết bị
    df['ua_is_bot'] = df['ua_parsed'].apply(lambda ua: int(ua.is_bot))
    df['ua_is_mobile'] = df['ua_parsed'].apply(lambda ua: int(ua.is_mobile))
    df['ua_is_tablet'] = df['ua_parsed'].apply(lambda ua: int(ua.is_tablet))
    df['ua_is_pc'] = df['ua_parsed'].apply(lambda ua: int(ua.is_pc))
    df['ua_is_touch_capable'] = df['ua_parsed'].apply(lambda ua: int(ua.is_touch_capable))

    # 11. Độ dài chuỗi User-Agent
    df['ua_length'] = df['user_agent'].astype(str).apply(len)

    # 12. Entropy của User-Agent
    df['ua_entropy'] = df['user_agent'].astype(str).apply(calculate_entropy)

    tools = ['sqlmap', 'curl', 'wget', 'nmap', 'nikto', 'fuzz', 'hydra']
    df['ua_is_tool'] = df['user_agent'].str.lower().apply(lambda ua: any(tool in ua for tool in tools)).astype(int)

    # Dọn bộ nhớ
    df.drop('ua_parsed', axis=1, inplace=True)

    return df

In [ ]:
df = user_agent_features(df)
df.sample(5)

In [ ]:
cols_to_show_ua = [
    'user_agent', 'ua_browser_family', 'ua_os_family', 'ua_device_brand',
    'ua_is_bot', 'ua_is_mobile', 'ua_is_pc', 'ua_length', 'ua_entropy', 'ua_'
]

print("\n--- Features từ User-Agent ---")
print(df[[col for col in cols_to_show_ua if col in df.columns]].head())

In [ ]:
def status_features(df):
    """
    Trích xuất các feature từ cột 'status' và 'size' trong log web.

    Args:
        df (pd.DataFrame): DataFrame chứa ít nhất 2 cột: 'status' (int), 'size' (int)

    Returns:
        pd.DataFrame: DataFrame gốc kèm thêm các cột đặc trưng mới.
    """
    # Kiểm tra cột trước
    if 'status' not in df.columns or 'size' not in df.columns:
        raise ValueError("DataFrame cần có cột 'status' và 'size'.")

    df = df.copy()
    df['status'] = pd.to_numeric(df['status'], errors='coerce').fillna(0).astype(int)
    df['size'] = pd.to_numeric(df['size'], errors='coerce').fillna(0).astype(int)
    # 1. 4xx - lỗi phía client
    df['status_is_client_error'] = df['status'].apply(lambda x: 1 if 400 <= x < 500 else 0)

    # 2. 5xx - lỗi phía server
    df['status_is_server_error'] = df['status'].apply(lambda x: 1 if 500 <= x < 600 else 0)

    # 3. Lỗi nói chung
    df['status_is_error'] = ((df['status_is_client_error'] == 1) | (df['status_is_server_error'] == 1)).astype(int)

    # 4. Thành công (2xx)
    df['status_is_success'] = df['status'].apply(lambda x: 1 if 200 <= x < 300 else 0)

    # 5. Redirect (3xx)
    df['status_is_redirect'] = df['status'].apply(lambda x: 1 if 300 <= x < 400 else 0)

    # 6. Response size bằng 0
    df['size_is_zero'] = df['size'].apply(lambda x: 1 if x == 0 else 0)

    return df


In [ ]:
df = status_features(df)
df.sample(5)

In [ ]:
df = referrer_features(df)
df.sample(3)

In [ ]:
# def behavior_features(df):
#     df = df.copy()

#     df['ip_request_count_total'] = df.groupby('ip')['ip'].transform('count')
#     df['ip_error_rate'] = df.groupby('ip')['status_is_error'].transform('mean')
#     df['ip_avg_query_count'] = df.groupby('ip')['url_query_count'].transform('mean')

#     return df


In [ ]:
# df = behavior_features(df)
# df.sample(3)

## Data preprocessing 2

In [ ]:
len(num_col)

In [ ]:
len(cat_col)

In [ ]:
cat_col

In [ ]:
# Đã trích xuất nên drop
#df.drop(columns=["ip", "url", "referrer"], inplace=True)
df.drop(columns=['user_agent'], inplace=True)

In [ ]:
df.drop(columns=["ip", "url", "referrer"], inplace=True)


In [ ]:
df.info()

In [ ]:
df.isna().sum()/len(df)

In [ ]:
df.drop(columns=['ua_device_brand', 'ua_browser_version_major', 'ua_os_version_major'], inplace=True)

In [ ]:

cat_col = df.select_dtypes(include=["object", "category"]).columns.tolist()

num_col = df.select_dtypes(include=["number"]).columns.tolist()


In [ ]:
# Cell 7: Áp dụng Feature Engineering
print("Processing Train set...")
train_featured = timestamp_features(train_df)
# train_featured['is_suspicious'] = train_featured['url'].apply(final_url_analyzer)
# train_featured = user_agent_features(train_featured)
train_featured = status_features(train_featured)
train_featured = referrer_features(train_featured)
# ... nếu có behavior_features, bạn phải fit trên train và transform cả hai ...


print("Processing Test set...")
test_featured = timestamp_features(test_df)
# test_featured['is_suspicious'] = test_featured['url'].apply(final_url_analyzer)
# test_featured = user_agent_features(test_featured)
test_featured = status_features(test_featured)
test_featured = referrer_features(test_featured)

In [ ]:
df.isna().sum()/len(df)

In [ ]:
df = df[~df['label'].isna()]

In [ ]:
# Cell 8: Chuẩn bị X, y
TARGET = 'label'
ua_cols_to_drop = [col for col in train_featured.columns if col.startswith('ua_')]

# Dọn dẹp các cột không cần thiết cho model
cols_to_drop = [
    'ip', 'url', 'referrer', 'user_agent', 
    'ua_browser_version_major', 'ua_os_version_major', 'ua_device_brand', 'is_suspicious', 'ua_is_tool', 'status_is_error'
] + ua_cols_to_drop

X_train = train_featured.drop(columns=[TARGET] + [col for col in cols_to_drop if col in train_featured.columns])
y_train = train_featured[TARGET]

X_test = test_featured.drop(columns=[TARGET] + [col for col in cols_to_drop if col in test_featured.columns])
y_test = test_featured[TARGET]

# Đảm bảo các cột trong X_train và X_test khớp nhau
X_test = X_test[X_train.columns]

# Cell 9: Huấn luyện CatBoost (giống như code của bạn)
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report

cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

model = CatBoostClassifier(
    verbose=100, 
    random_state=42,
    # Thêm các tham số chống overfitting nếu cần
    # auto_class_weights='Balanced', # Thử cái này nếu dữ liệu mất cân bằng
    # early_stopping_rounds=50 
)

# Để dùng early_stopping_rounds, bạn cần fit với eval_set
# model.fit(train_pool, eval_set=test_pool) 
model.fit(train_pool) # Hoặc fit như cũ

# Đánh giá
y_pred = model.predict(test_pool)
print(classification_report(y_test, y_pred))

In [ ]:
print("\\n--- Feature Importance ---")

# Lấy độ quan trọng
feature_importances = model.get_feature_importance()
feature_names = X_train.columns

# Tạo DataFrame để dễ xem
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

print(importance_df.head(20)) # In ra 20 feature quan trọng nhất

# Vẽ biểu đồ
plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=importance_df.head(20))
plt.title('Top 20 Feature Importances')
plt.tight_layout()
plt.show()

In [ ]:
df.columns

In [ ]:
import joblib
import json

# Giả sử bạn vừa train xong CatBoost model:
# model = CatBoostClassifier(...)
# model.fit(X_train, y_train)

# Cấu hình lưu kèm: các cột đã dùng và feature categorical
model_columns = X_train.columns.tolist()
categorical_features = [col for col in model_columns if str(X_train[col].dtype) == 'category']

# --- Lưu model ---
joblib.dump(model, '/kaggle/working/catboost_model.joblib')

# --- Lưu cấu hình (các cột & categorical) ---
config = {
    'model_columns': model_columns,
    'categorical_features': categorical_features,
}

with open('/kaggle/working/model_config.json', 'w') as f:
    json.dump(config, f)

print("✅ Đã lưu model và cấu hình vào /kaggle/working/")
